EE488 HW #1: Classification of MNIST Dataset
====

## Instruction
- In this HW assignment, we design a MNIST classifier using convolutional neural network
- Complete the code by implementing the architecture described in "CNN Architecture" section.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as vision_dsets
import torchvision.transforms as T
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data
import matplotlib.pyplot as plt

## Initilaize Data Loader

In [ ]:
root='./data'
batch_size=32
# Use Mnist data for training

mnist_train = vision_dsets.MNIST(root = root,
                                train = True,
                                transform = T.ToTensor(),
                                download = True)
mnist_test = vision_dsets.MNIST(root = root,
                                train = False,
                                transform = T.ToTensor(),
                                download = True)

trainDataLoader = data.DataLoader(dataset = mnist_train,
                                    batch_size = batch_size,
                                    shuffle =True,
                                    num_workers = 1)

testDataLoader = data.DataLoader(dataset = mnist_test,
                                batch_size = batch_size,
                                shuffle = False,
                                num_workers = 1)


## Define Trainer

In [ ]:
class Trainer():
    def __init__(self, trainloader, testloader, net, optimizer, criterion):

        self.trainloader = trainloader
        self.testloader = testloader
        self.net = net
        self.optimizer = optimizer
        self.criterion = criterion

    def train(self, epoch = 1, log_freq = 500):
        self.net.train()
        for e in range(epoch):
            running_loss = 0.0
            for i, (inputs, labels) in enumerate(self.trainloader, 0):
                inputs = inputs.cuda()
                labels = labels.cuda()
                self.optimizer.zero_grad()

                outputs = self.net(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                # print statistics
                running_loss += loss.item()
                if (i+1) % log_freq == 0:
                    print('[%d, %5d] loss: %.3f' % (e + 1, i + 1, running_loss / log_freq))
                    running_loss = 0.0

        print('Finished Training')

    def test(self):
        self.net.eval()

        test_loss = 0
        correct = 0
        for inputs, labels in self.testloader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            output = self.net(inputs)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(labels.view_as(pred)).sum().item()

            test_loss /= len(self.testloader.dataset)
        print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.
                format(correct, len(self.testloader.dataset),
                100.* correct / len(self.testloader.dataset)))

# CNN Architecture
Complete this section to implenent a classifier with 3 CNN layers and 2 FCN layers.

Details of architecture are provided in comments below:

In [ ]:
## Fill in the empty space
# Architecture: 3-layers of CNN, 2-layers of FCN
# Layer 1: convolutional layer + batch norm 2d + relu activation
#          output channels: 32, kernel size: (2,2), stride: (2,2)
# Layer 2: convolutional layer + batch norm 2d + relu activation
#          output channels: 64, kernel size: (2,2), stride: (2,2)
# Layer 3: convolutional layer + batch norm 2d + relu activation
#          output channels: 128, kernel size: (2,2), stride: (2,2)
# Layer 4: fully connected layer
#          input: flattened 1d vector from Layer 3 output, output channels: 512
# Layer 5: fully connected layer: output channels 10 (=number of classes)

class MNIST_Net(nn.Module):
    def __init__(self, hidden1, hidden2, hidden3):
        super(MNIST_Net, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=hidden1, kernel_size=(2, 2), stride=(2, 2)),
            nn.BatchNorm2d(hidden1),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden1, out_channels=hidden2, kernel_size=(2, 2), stride=(2, 2)),
            nn.BatchNorm2d(hidden2),
            nn.ReLU()
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=hidden2, out_channels=hidden3, kernel_size=(2, 2), stride=(2, 2)),
            nn.BatchNorm2d(hidden3),
            nn.ReLU()
        )

        # Assume MNIST input size: (1, 28, 28)
        # Output size after 3 conv layers with stride (2,2) and kernel size (2,2) = (128, 3, 3)
        self.fc1 = nn.Linear(hidden3 * 3 * 3, 512)
        self.fc2 = nn.Linear(512, 10)

        # Fill in here

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [ ]:
mnist_net = MNIST_Net(hidden1= 32, hidden2 = 64, hidden3 = 128).cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001)

In [ ]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [ ]:
trainer.train(epoch = 1)

In [ ]:
trainer.test()

# Plotting
Below code plots figure of the original input image and predicted probability for each class.

In [ ]:
n = 10 # the number of data to be plotted
plt.figure(figsize=(10, 20)) # total figure size

for i in range(n):
    input_image = mnist_test[i][0].cuda() # (1,28,28) tensor
    input_ = input_image.unsqueeze(0)     # change to (1,1,28,28)
    pred_prob = torch.softmax(mnist_net(input_),dim=1).cpu().detach().squeeze().numpy()

    ax = plt.subplot(n, 2, 2*i+1)
    plt.imshow(input_image[0].cpu()) # show input image
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    ax2 = plt.subplot(n, 2, 2*i+2)
    ax2.set_ylim([0,1])

    classify_as = [str(i) for i in range(n)]
    plt.bar(classify_as, pred_prob, width = 0.5, tick_label = classify_as) # print predicted probability for each class

    plt.title('predicted probability')
    plt.tight_layout()

plt.show()